In [ ]:
# One-time setup: run this cell, then restart the kernel if needed
! pip install -q "transformers" "datasets" "protobuf<5"


In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = "Hello, how are you?"
# tokenize and encode the text
encoding = tokenizer.encode(text)
print("Token IDs:", encoding)

# convert token ids back to text
decoded_text = tokenizer.decode(encoding)
print("Decoded Text:", decoded_text)


In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForNextSentencePrediction, Trainer, TrainingArguments
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
